In [1]:
from ScreeningFactorData import CompositionData, ScreeningFactorData
from ScreeningFactorNetwork import ScreeningFactorNetwork
import pynucastro as pyna
import numpy as np

In [2]:
nuclei = ["h1", "he4", "c12", "o16", "n14", "ca40"]
num_nuclei = len(nuclei)
comp = pyna.Composition([pyna.Nucleus(n) for n in nuclei])

reactants = ["c12", "he4"]

temp_range = (1.e7, 1.e10)
dens_range = (1.e4, 1.e8)

input_kwargs = {
    "temperature_range": temp_range,
    "density_range": dens_range,
    "num_nuclei": num_nuclei,
    "size": 10**5
}

output_kwargs = {
    "comp": comp,
    "reactants": reactants,
    "threshold": 1.01
}

In [3]:
training = ScreeningFactorData(
    inputs=CompositionData(**input_kwargs, seed=0),
    **output_kwargs
)
testing = ScreeningFactorData(
    inputs=CompositionData(**input_kwargs, seed=1),
    **output_kwargs
)

In [4]:
training.inputs.x

array([[0.63696169, 0.60699537, 0.10167085, ..., 0.28562701, 0.0544318 ,
        0.12996116],
       [0.26978671, 0.21254401, 0.10483623, ..., 0.44504448, 0.0438048 ,
        0.01901465],
       [0.04097352, 0.55624315, 0.1967304 , ..., 0.32866369, 0.05964874,
        0.35713564],
       ...,
       [0.77706236, 0.05828959, 0.27655458, ..., 0.16259219, 0.07665661,
        0.0426203 ],
       [0.57903921, 0.73793846, 0.23899268, ..., 0.04268075, 0.23729338,
        0.09504649],
       [0.50626261, 0.44679832, 0.14992474, ..., 0.01028611, 0.29169643,
        0.12387776]])

In [5]:
network = ScreeningFactorNetwork(
    training_data=training,
    testing_data=testing,
    seed=0
)

In [6]:
network.fit_model(verbose=2)

Epoch 1/20
425/425 - 3s - 7ms/step - accuracy: 0.9723 - loss: 0.0688 - val_accuracy: 0.9933 - val_loss: 0.0250
Epoch 2/20
425/425 - 2s - 5ms/step - accuracy: 0.9858 - loss: 0.0327 - val_accuracy: 0.9877 - val_loss: 0.0246
Epoch 3/20
425/425 - 2s - 5ms/step - accuracy: 0.9881 - loss: 0.0275 - val_accuracy: 0.9947 - val_loss: 0.0164
Epoch 4/20
425/425 - 2s - 5ms/step - accuracy: 0.9895 - loss: 0.0249 - val_accuracy: 0.9955 - val_loss: 0.0139
Epoch 5/20
425/425 - 2s - 5ms/step - accuracy: 0.9901 - loss: 0.0226 - val_accuracy: 0.9970 - val_loss: 0.0118
Epoch 6/20
425/425 - 2s - 5ms/step - accuracy: 0.9907 - loss: 0.0213 - val_accuracy: 0.9964 - val_loss: 0.0117
Epoch 7/20
425/425 - 2s - 5ms/step - accuracy: 0.9910 - loss: 0.0203 - val_accuracy: 0.9972 - val_loss: 0.0110
Epoch 8/20
425/425 - 2s - 5ms/step - accuracy: 0.9916 - loss: 0.0198 - val_accuracy: 0.9981 - val_loss: 0.0103
Epoch 9/20
425/425 - 2s - 6ms/step - accuracy: 0.9920 - loss: 0.0189 - val_accuracy: 0.9981 - val_loss: 0.0106
E

In [7]:
network.predict(
    temp=1.e8,
    dens=1.e6,
    mass_frac=np.ones(6)/6
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


array([[9.187461e-20, 1.000000e+00]], dtype=float32)

In [8]:
network.predict(
    temp=[1.e8, 1.e9],
    dens=[1.e6, 1.e7],
    mass_frac=testing.inputs.rng.dirichlet(alpha=[1]*num_nuclei, size=(2,))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


array([[4.6579681e-20, 1.0000000e+00],
       [6.6265685e-14, 1.0000000e+00]], dtype=float32)

In [16]:
input_kwargs_2 = input_kwargs
input_kwargs_2["num_nuclei"] = [1, 2, 3, 5, 1, 2]

test2 = CompositionData(**input_kwargs_2, seed=2)

display(
    test2.x,
    network.model.predict(test2.x)
)

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


array([[0.26161213, 0.77217487, 0.07142857, ..., 0.35714286, 0.07142857,
        0.14285714],
       [0.29849114, 0.0677453 , 0.07142857, ..., 0.35714286, 0.07142857,
        0.14285714],
       [0.81422574, 0.75680663, 0.07142857, ..., 0.35714286, 0.07142857,
        0.14285714],
       ...,
       [0.35486088, 0.8514249 , 0.07142857, ..., 0.35714286, 0.07142857,
        0.14285714],
       [0.21557881, 0.41608896, 0.07142857, ..., 0.35714286, 0.07142857,
        0.14285714],
       [0.27102194, 0.90353063, 0.07142857, ..., 0.35714286, 0.07142857,
        0.14285714]])

array([[3.8706370e-25, 1.0000000e+00],
       [1.0847571e-15, 1.0000000e+00],
       [1.2901570e-09, 1.0000000e+00],
       ...,
       [1.6462951e-23, 1.0000000e+00],
       [4.0989261e-22, 1.0000000e+00],
       [1.7912996e-26, 1.0000000e+00]], dtype=float32)